# Partial Differential Equations (PDEs)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation, cm
from scipy import integrate, linalg, sparse

## Introduction

A **partial differential equation, or PDE**, is an equation involving partial derivatives of an unknown function with respect to more than one independent variable.
PDEs are of fundamental importance in modeling all types of continuous phenomena in nature. 

Many of the basic laws of science are expressed as PDEs, including:

- Maxwell’s equations, which describe the behavior of an electromagnetic field by prescribing the relationships among the electric and magnetic field strengths, magnetic flux density, and electric charge and current densities.

$$
\begin{aligned}
\nabla\cdot D &= \frac{\rho}{\epsilon_0} \\
\nabla\cdot B &= 0 \\
\nabla\times E &= -\frac{\partial B}{\partial Bt} \\
\nabla\times B &= \mu_0 J+\frac{\partial E}{c^2\partial Bt}
\end{aligned}
$$

- Navier-Stokes equations, which describe the behavior of a fluid by prescribing the relationships among its velocity, density, pressure, and viscosity.

$$
\begin{aligned}
\nabla\cdot u &=0 \\
\rho \frac{\partial u}{\partial t} &= -\nabla p+\mu\nabla^2 u+\rho F
\end{aligned}
$$

- Schrödinger’s equation of quantum mechanics, which describes the wave function of a particle by prescribing the relationships among its mass, potential energy, and total energy.

$$i\hbar\frac{\partial}{\partial t} \Psi =\left[\frac{-\hbar^2}{2m}\nabla^2+V\right]\Psi$$

- Einstein’s equations of general relativity, which describe a gravitational field by prescribing the relationship between the curvature of spacetime and the energy density of the matter it contains.

- Linear elasticity equations, which describe vibrations in an elastic solid with given material properties by prescribing the relationship between stress and strain.

We will confine our attention to PDEs that are simpler than those just listed, as the general theory of PDEs is far beyond the scope of this course. 

We will consider some basic concepts and methods in relatively simple settings, but most of these are applicable more generally. 
Many of these ideas carry over from ODEs, such as the need to specify initial or boundary conditions, but the situation is typically more complex with PDEs, in part because a problem domain in two or more dimensions can be much more irregular, and the boundary conditions much more complicated, than in one dimension. 

Many of the numerical solution techniques we saw in the ODE notebook also carry over to PDEs, but the  computational cost increases substantially with the number of independent variables because the
system of algebraic equations resulting from discretization becomes much larger.

### Notation

For simplicity, we will deal only with single PDEs (as opposed to systems of several coupled PDEs) with only two
independent variables:

- One space variable denoted by $x$ and a time variable denoted by $t$ (analogous to the initial value problems for ODEs)
- Two space variables denoted by $x$ and $y$ (analogous to the boundary value problems for ODEs).

We denote the unknown solution function by $u$, and we denote its partial derivatives with respect to the independent variables by appropriate subscripts: 

- $u_t = \partial u/\partial t$
- $u_{xy} =\partial^2 u /\partial x\partial y$
- $\cdots$

We seek to determine a function $u$ whose partial derivatives with respect to the independent variables satisfy the relationship prescribed by a given PDE on a given domain, and which also satisfies whatever initial or boundary conditions may have been imposed. 

Such a solution function u can be visualized as a surface over the relevant two-dimensional domain in the $(t, x)$ or $(x, y)$ plane.

## Classification and examples

As with ODEs, the order of a PDE is determined by the highest-order partial derivative appearing in the PDE.

Some of the most important PDEs in practical applications are of second order, including
- **Heat equation**

$$u_t = u_{xx}$$

- **Wave equation**

$$u_{tt} = u_{xx}$$

- **Laplace equation**

$$u_{xx} + u_{yy} = 0$$

It turns out that these three equations are general prototypes in that any second-order linear PDE of the form

$$a u_{xx} + b u_{xy} + c u_{yy} + d u_x + e u_y + f u + g = 0$$

can be transformed by a change of variables into one of these three canonical equations (plus terms of lower order), provided $a$, $b$, and $c$ are not all zero. 

The quantity $b^2 - 4ac$, called the discriminant, determines which of the canonical forms is obtained by such a transformation. 

Therefore, second-order linear PDEs can be classified according to the value of the discriminant into three families whose names derive from the analogous conic sections:

- $b^2 - 4ac > 0$: **hyperbolic**, typified by the wave equation
- $b^2- 4ac = 0$: **parabolic**, typified by the heat equation
- $b^2 - 4ac < 0$: **elliptic**, typified by the Laplace equation


> **Where do these names come from?**
>
> The equation
>
> $$a u_{xx} + b u_{xy} + c u_{yy} + d u_x + e u_y + f u + g = 0$$
>
> is reminiscent of the equation
>
> $$a x^2+b xy+c y^2 +d x +e y$$
>
> which is the general form for *conic sections*,
> where the prototypical hyperbola, parabola and ellipse are given by
>
> $$
\begin{aligned}
x^2-y^2 &= 1 \\
y &= x^2 \\
x^2+y^2 &= 1
\end{aligned}
$$

Generally,
- *Hyperbolic* PDEs describe time-dependent, conservative physical processes, such as convection, that are not evolving toward a steady state.
-  *Parabolic* PDEs describe time-dependent, dissipative physical processes, such as diffusion, that are evolving toward a steady state.
- *Elliptic* PDEs describe systems that have already reached a steady state, or equilibrium, and hence are time-independent.

Systems governed by *hyperbolic* PDEs are conservative in that the “energy” of the system, as measured by an appropriate norm of the solution, is conserved over time. 
Hyperbolic PDEs are analogous to a linear system of ODEs whose matrix has purely imaginary eigenvalues, yielding a purely oscillatory solution that neither grows nor decays with time. 

Systems governed by *parabolic* PDEs, on the other hand, are dissipative in that the “energy” of the solution diminishes over time.
Parabolic PDEs are analogous to a linear system of ODEs whose matrix has only eigenvalues with negative real parts, yielding an exponentially decaying solution.

Another important difference is that hyperbolic PDEs propagate information at a finite speed, whereas parabolic PDEs propagate information instantaneously.

These differences between the two types of time-dependent PDEs have important theoretical and practical implications. 

For example, parabolic PDEs have a smoothing effect that over time damps out any lack of smoothness in the initial
conditions, whereas hyperbolic PDEs propagate steep fronts or shocks undiminished, and discontinuities can develop in the solution even with smooth initial data. 

Systems governed by hyperbolic PDEs are in principle reversible in time, whereas parabolic systems are not. 
The heat equation integrated backward in time is ill-posed, for example, which corresponds physically to the fact that one cannot determine details of the thermal history of a system from its current temperature distribution. 

The challenges in solving parabolic or hyperbolic PDEs numerically are analogous to those in solving ODEs that are stiff because of eigenvalues with large negative real parts (parabolic) or large imaginary parts (hyperbolic).

## Solving time-dependent problems

Numerical methods for time-dependent PDEs typically use discrete time-stepping procedures to generate an approximate solution step-by-step in time, analogous to the methods for ODE initial value problems.
The corresponding spatial discretization can be accomplished in a number of ways

The 2 most common are:

- **Semidiscrete methods**

> One way to approximate the solution to a time-dependent PDE numerically is to discretize in space but leave the time variable continuous. This approach results in a system of ODEs, which can then be solved by the methods discussed in the ODE notebook.

- **Fully discrete methods**

> In a fully discrete method, all of the independent variables in the PDE are discretized, including time. 
>
>In a fully discrete finite difference method, we introduce a grid of mesh points throughout the problem domain in space and time, we replace all the derivatives in the PDE by finite difference approximations, and we seek an
approximate value for the solution at each of the mesh points. 
The resulting array of approximate solution values represents a discrete sample of points on the solution surface over the problem domain in the $(t, x)$ plane. 
The accuracy of such an approximate solution depends on the step sizes in both space and time.
>
>Replacement of all partial derivatives by finite difference approximations results in a system of algebraic equations for the unknown solution values at the discrete set of mesh points. 
>
>This system may be linear or nonlinear, depending on the underlying PDE. 
With an initial-value problem, the solution is obtained by beginning with the initial values along an appropriate boundary of the problem domain and then marching forward step by step in time, generating successive rows in the solution array. 
>
>Such a time-stepping procedure may be **explicit** or **implicit**, depending on whether the formula for the solution values at the next time step involves only current and past information.

Practical examples of both discretization methods are shown below for specific examples of the Heat and Wave equation.

### Heat equation

The heat equation in one space dimension has the form

$u_t = c u_{xx}$, $0\leq x\leq L$, $t\geq 0$, 

with given initial condition $u(0,x)=f(x)$, $0\leq x\leq L$,

and boundary conditions $u(t,0)=\alpha$, $u(t,L)=\beta$ , $t\geq 0$.

and $c$ a positive constant. 

This equation models, for example, the diffusion of heat in a bar of length $L$ whose ends are maintained at temperatures specified by the boundary conditions and whose initial temperature distribution is given by the function $f(x)$. 
The constant $c$, which governs the rate of diffusion, depends on material properties of the bar, such as its thermal conductivity, specific heat, and density. 

The solution $u$ to this equation gives the subsequent temperature distribution as a function of both space and time

> **Example: solving the heat equation with a semidiscrete method**
>
> Consider the equation, describing the diffusion of heat in a 1 m long bar, with one end kept at 0 K, and the other at 1 K.
> 
> The initial temperature distribution is given by the superposition of a straight line and a Gaussian curve $f(x)=x+\exp{-\left(\frac{(x-0.5)}{0.1}\right)^2}$ 
> 
> $u_t = c u_{xx}$, $0\leq x\leq 1$, $t\geq 0$, 
>
> with given initial condition $u(0,x)=f(x)$, $0\leq x\leq L$,
>
> and boundary conditions $u(t,0)=0$, $u(t,1)=1$ , $t\geq 0$.
>
> with $c$ the [thermal diffusivity](https://en.wikipedia.org/wiki/Thermal_diffusivity) of the material (e.g. $23\,\mathrm{mm^2/s}$ for iron).
>
>
> If we introduce spatial mesh points $x_i = i\Delta x$, $i = 0,\ldots, n+1$, where $\Delta x = 1/(n+1)$, and replace the second derivative $u_{xx}$ with the finite difference approximation
>
> $$u_{xx}(t,x_i)\approx\frac{u(t,x_{i+1})-2u(t,x_i)+u(t,x_{i-1})}{(\Delta x)^2}\quad,\quad i=1,\ldots,n$$
>
> but leave the time variable continuous, then we obtain a system of ODEs
>
> $$y_i'(t)=\frac{c}{(\Delta x)^2}(y_{i+1}(t)-2y_i(t)+y_{i-1}(t))\quad,\quad i=1,\ldots,n$$
>
> where $y_i (t) \approx u(t, x_i )$.
> From the boundary conditions we know that, $y_0(t)=0$ and $y_{n+1}(t)=1 $, and from the initial conditions, that $y_i (0) = f (x_i )$, $i = 1,\ldots, n$
>
>We can therefore use an ODE method to solve the initial value problem for this system.
>
> This approach is sometimes called **the method of lines**.
>If we think of the solution $u(t, x)$ as a surface over the $(t, x)$ plane, this method computes cross sections of
that surface along a series of lines, each of which passes through one of the discrete spatial mesh points and runs parallel to the time axis.
>
> The foregoing semidiscrete system of ODEs can be written in matrix form as
>
> $$
\mathbf{y}'=\frac{c}{(\Delta x)^2}\begin{bmatrix}
-2& 1& 0&\cdots&0\\ 
1& -2& 1&\cdots&0\\ 
0& 1& -2&\cdots&0\\ 
\vdots&\ddots&\ddots&\ddots&\vdots\\
0&\cdots&0&1&-2\end{bmatrix}\mathbf{y}=\mathbf{Ay}
$$
>
> The Jacobian matrix $\mathbf{A}$ of this system has eigenvalues between $-4c/(\Delta x)^2$ and 0, which makes the ODE very stiff as the spatial mesh size $\Delta x$ becomes small.
> This stiffness, which is typical of ODEs derived from PDEs in this manner, must be taken into account in choosing an appropriate ODE method for solving the semidiscrete system.

In [ ]:
def demo_heat_equation():
    """Define and solve heat equation example with a semidiscrete method."""
    # define spatial discretization and thermal diffusion constant
    N = 100
    c = 23e-6

    # define A
    A = (
        sparse.diags(np.ones(N - 1), -1).toarray()
        + sparse.diags(np.ones(N) * -2, 0).toarray()
        + sparse.diags(np.ones(N - 1), 1).toarray()
    )
    print(A)
    A = A * c * (N + 1) ** 2

    # boundary conditions: do not change edge points
    A[0, :] = 0
    A[N - 1, :] = 0

    # y'=Ay (nice and simple once you have A)
    def func(t, y):
        return A @ y

    # Discretize our rod
    x = np.linspace(0, 1, N)
    # Define initial temperature profile and plot it
    y0 = np.exp(-1 * ((x - 0.5) / 0.1) ** 2) + x

    # Use an ODE solver to solve this problem
    sol = integrate.solve_ivp(func, [0, 2500], y0, t_eval=np.arange(0, 2501, 10))

    # --- Everything below makes an animation of the solution ---

    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(num="heat", clear=True)
    ax = plt.axes(xlim=(0, 1), ylim=(0, 1.5))
    ax.plot(x, y0, ".", label="initial")
    (line,) = ax.plot([], [], lw=2, label="time-dependent")
    ax.legend()

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return (line,)

    # animation function.
    def animate(i):
        x = np.linspace(0, 1, N)
        y = sol.y[:, i]
        line.set_data(x, y)
        return (line,)

    # call the animator.  blit=True means only re-draw the parts that have changed.
    return animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=250,
        interval=20,
        blit=True,
        repeat=False,
    )


plt.close("heat")
demo_heat_equation()

### Wave equation

The wave equation in one spatial dimension has the form

$u_{tt} = c u_{xx}$ , $0 \leq x \leq L$, $t \geq 0$,

with given initial conditions $u(0, x) = f (x)$ , $u_t(0, x) = g(x)$, $0 \leq x \leq L$,

and boundary conditions $u(t, 0) = \alpha$ , $u(t, L) = \beta$, $t \geq 0$,

and $c$ a positive constant. 

This equation models, for example, vibrations of a violin string of length $L$ whose initial profile and velocity are given by the functions $f(x)$ and $g(x)$, respectively, and whose ends are anchored as prescribed by the boundary conditions. 

Because it is second-order in time, this equation requires initial conditions for both the solution function and its first derivative with respect to time. 
The solution consists of waves propagating both to the left and the right with speed $\sqrt{c}$.

> **Example: solving the wave equation with a fully discrete method**
>
>
> We define spatial mesh points $x_i = i\Delta x$, $i = 0, 1,\ldots , n + 1$, where $\Delta x = L/(n + 1)$,
and temporal mesh points $t_k = k \Delta t$, $k = 0, 1, \ldots $, where $\Delta t$ is chosen appropriately.
>
> We denote the approximate solution at mesh point $(t_k , x_i)$ by $u^k_i$ , where we have used both a subscript and a superscript (the $k$ is not an exponent) to distinguish clearly between increments in space and time, respectively.
>
> Using centered difference approximations for both $u_{tt}$ and $u_{xx}$ yields a system of algebraic equations
>
>$$\frac{u_i^{k+1}-2u_i^k+u_i^{k-1}}{(\Delta t)^2}=c \frac{u^k_{i+1}-2u^k_{i}+u^k_{i-1}}{(\Delta x)^2}\quad,\quad i=1,\ldots,n$$ 
>
> which can be rearranged to give the explicit recurrence
>
>$$u_i^{k+1}=2u_i^k-u_i^{k-1}+c\left(\frac{\Delta t}{\Delta x}\right)^2 (u^k_{i+1}+2u^k_i+u^k_{i-1})$$
>

The pattern of mesh points involved in computing $u_i^{k+1}$ is illustrated in the figure below, where lines connect the relevant mesh points and an arrow indicates the mesh point at which the approximate solution is being computed. 
Such a pattern is called the **stencil** of a given finite difference scheme.

In [ ]:
def plot_stencil_1():
    """Plot the stencil for the explicit method for the wave equation."""
    plt.close("stencil1")
    fig, ax = plt.subplots(num="stencil1")
    ax.plot(np.arange(3), np.zeros(3), "o", c="k")
    ax.plot(np.arange(3), np.ones(3), "o", c="k")
    ax.plot(np.arange(3), 2 * np.ones(3), "o", c="k")
    ax.text(-0.5, 0, "k-1", fontsize="xx-large")
    ax.text(-0.5, 1, "k", fontsize="xx-large")
    ax.text(-0.5, 2, "k+1", fontsize="xx-large")

    ax.text(0, -0.5, "i-1", fontsize="xx-large")
    ax.text(1, -0.5, "i", fontsize="xx-large")
    ax.text(2, -0.5, "i+1", fontsize="xx-large")

    ax.arrow(1, 0, 0, 1.75, fc="k", ec="k", head_width=0.10, head_length=0.25)

    ax.axis("off")

    ax.plot([0, 2], [1, 1], c="k")
    ax.plot([1, 1], [0, 2], c="k")


plot_stencil_1()

 This scheme is second-order accurate in both space and time, but it requires data at two successive time levels, which means that additional storage is required, and it also means that we need both $u^0_i$ and $u^1_i$ initially to get started. 
 These values can be obtained from the initial conditions
 $u^0_i = f(x_i)$, $u^1_i = u^0_i + \Delta t g(x_i)$ , $i = 1, \ldots, n$, where in the latter we have used a forward difference approximation to the initial condition $u_t(0, x) = g(x)$, $0 \leq x \leq 1$.

> To make this more specific, let's consider the wave equation
>
> $$\frac{\partial^2f(x,t)}{\partial x^2}=\frac{\rho}{T}\frac{\partial^2f(x,t)}{\partial t^2}$$
> 
> or, in our notation,
> 
> $$u_{xx}=\frac{\rho}{T}u_{tt}$$
>
> describing the waves on a string with a density $\rho$ (in kg/m) under tension $T$ (in N).
> 
> with the boundary conditions that the string is attached at $x=0$ and $x=1$.

In [ ]:
def check_stability_criterion(c, delta_x, delta_t):
    """Check if the finite difference method is stable.

    This function is used by `demo_wave_equation()` below.

    Parameters
    ----------
    c : float
        The speed of the wave
    delta_x : float
        The distance between each spatial point
    delta_t : float
        The time step

    Returns
    -------
    bool
        True if the method is stable, False otherwise
    """
    stability_criterion = c * (delta_t / delta_x) ** 1
    if stability_criterion > 1:
        print("Warning: The method is unstable. Consider using smaller time steps.")
    return stability_criterion <= 1


def demo_wave_equation():
    """Define and solve a wave equation example with a finite-difference method."""

    # define spatial discretization
    N = 100
    c = 1

    # define our 1D-grid
    x = np.linspace(0.0, 1.0, N)
    delta_x = x[1] - x[0]

    # we will model a standing wave where the displacement
    # is given analytically by  y=sin(2 pi x)cos(2 pi ct)
    # at time t=0 we get
    y = np.sin(x * 2 * np.pi)
    # which has dy/dt= -2*pi*sin(2 p i x) as velocity
    g = -2.0 * np.pi * np.sin(x * 2 * np.pi)

    # time "grid"
    t_range = np.linspace(0, 2, 200)
    delta_t = t_range[1] - t_range[0]

    # define A
    A = (
        sparse.diags(np.ones(N - 1), -1).toarray()
        + sparse.diags(np.ones(N) * -2, 0).toarray()
        + sparse.diags(np.ones(N - 1), 1).toarray()
    )
    A = A * c * (delta_t / delta_x) ** 2

    check_stability_criterion(c, delta_x, delta_t)

    # boundary conditions: do not change edge points
    A[0, :] = 0
    A[N - 1, :] = 0

    # first step using y'
    yprev = y
    y = y + delta_t * g

    # initialize an empty array in which we'll store the solutions
    solutions = np.zeros((len(t_range) - 1, N))

    # Do the actual work.
    # Note the helper variable 'ybetween' because
    # we need information of a previous time point.
    for t in range(len(t_range) - 1):
        ybetween = y
        y = 2 * y - yprev + A @ y
        solutions[t, :] = y
        yprev = ybetween

    # --- Everything below makes an animation of the solution ---

    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(num="wave", clear=True)
    ax = plt.axes(xlim=(0, 1), ylim=(-4, 4))
    (line,) = ax.plot([], [], lw=2)

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return (line,)

    # animation function.
    def animate(i):
        x = np.linspace(0, 1, N)
        y = solutions[i, :]
        line.set_data(x, y)
        return (line,)

    # call the animator.  blit=True means only re-draw the parts that have changed.
    return animation.FuncAnimation(
        fig,
        animate,
        init_func=init,
        frames=(len(t_range) - 1),
        interval=20,
        blit=True,
        repeat=False,
    )


plt.close("wave")
demo_wave_equation()

In principle, there is no real distinction between discrete and semidiscrete methods for time-dependent PDEs, since the time variable is ultimately discretized in either case. 
There is an important practical distinction, however, in that with a semidiscrete method we entrust to a sophisticated, adaptive ODE software package the responsibility for choosing time step sizes that will maintain stability and attain the desired accuracy, whereas with a fully discrete method, the user must explicitly choose time step sizes to achieve these same goals.

### Implicit methods: heat equation revisited

As with ODEs, a larger stability region that permits larger time steps can be obtained by using implicit methods. For the heat equation, for example, applying the backward Euler method to the semidiscrete system shown in the Heat equation example yields the implicit finite difference scheme

$$
u_i^{k+1}=u^k_i+c\frac{\Delta t}{(\Delta x)^2}(u^{k+1}_{i+1}-2u_i^{k+1}+u^{k+1}_{i-1})\quad,\quad i=1,\ldots,n
$$

whose stencil is shown here: 

In [ ]:
def plot_stencil_2():
    """Plot the stencil for the implicit method for the heat equation."""
    plt.close("stencil2")
    fig, ax = plt.subplots(num="stencil2")
    ax.plot(np.arange(3), np.zeros(3), "o", c="k")
    ax.plot(np.arange(3), np.ones(3), "o", c="k")
    ax.plot(np.arange(3), 2 * np.ones(3), "o", c="k")
    ax.text(-0.5, 0, "k-1", fontsize="xx-large")
    ax.text(-0.5, 1, "k", fontsize="xx-large")
    ax.text(-0.5, 2, "k+1", fontsize="xx-large")

    ax.text(0, -0.5, "i-1", fontsize="xx-large")
    ax.text(1, -0.5, "i", fontsize="xx-large")
    ax.text(2, -0.5, "i+1", fontsize="xx-large")

    ax.arrow(1, 1, 0, 0.75, fc="k", ec="k", head_width=0.10, head_length=0.25)

    ax.axis("off")

    ax.plot([0, 2], [2, 2], c="k")
    ax.plot([1, 1], [1, 2], c="k")


plot_stencil_2()

In the demo below we again solve the heat equation described in section 3.1, but now using the implicit method whose stencil is shown above.

In [ ]:
def demo_heat_equation_implicit():
    """Define and solve a heat equation example with an implicit method."""
    # define spatial and temporal discretizaiton
    Nx = 100
    Nt = 1000
    c = 23e-6

    # define A
    A = (
        sparse.diags(np.ones(Nx - 1), -1).toarray()
        + sparse.diags(np.ones(Nx) * -2, 0).toarray()
        + sparse.diags(np.ones(Nx - 1), 1).toarray()
    )
    A = A * c * (Nx + 1) ** 2

    # boundary conditions: do not change edge points
    A[0, :] = 0
    A[Nx - 1, :] = 0

    # y'=Ay (nice and simple once you have A)
    def func(t, y):
        return A @ y

    # Discretize our rod
    x = np.linspace(0, 1, Nx)
    # Define initial temperature profile and plot it
    y0 = np.exp(-1 * ((x - 0.5) / 0.1) ** 2) + x

    # initialize an empty array in which we'll store the solutions
    sol = np.zeros((Nt, Nx))

    y = y0
    # do the actual work.
    for t in np.arange(Nt):
        # the equation we're trying to solve is written implicitly as
        # ynext = y+A@ynext
        # this can be written in the form Ax=b as [A-1]ynext=-y
        # which can then be solved using linalg.solve:

        ynext = linalg.solve(A - sparse.diags(np.ones(Nx), 0).toarray(), -1 * y)

        sol[t, :] = ynext
        y = ynext

    # ---Everything below makes an animation of the solution ---

    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(num="heat_implicit", clear=True)
    ax = plt.axes(xlim=(0, 1), ylim=(0, 1.5))
    ax.plot(x, y0, ".", label="initial")
    (line,) = ax.plot([], [], lw=2, label="time-dependent")
    ax.legend()

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        return (line,)

    # animation function.
    def animate(i):
        x = np.linspace(0, 1, Nx)
        y = sol[i, :]
        line.set_data(x, y)
        return (line,)

    # call the animator.  blit=True means only re-draw the parts that have changed.
    return animation.FuncAnimation(
        fig, animate, init_func=init, frames=Nt, interval=2, blit=True, repeat=False
    )


plt.close("heat_implicit")
demo_heat_equation_implicit()

The scheme used above inherits the unconditional stability of the backward Euler method, which means that there is no stability restriction on the relative sizes of $\Delta t$ and $\Delta x$.

Accuracy is still a consideration, however, and the fact that this particular method is only first-order accurate in time still strongly limits the time step. 

If instead we apply the trapezoid method we obtain the implicit finite difference scheme

$$
u^{k+1}_i=u_i^k+c\frac{\Delta t}{2(\Delta x)^2}(u_{i+1}^{k+1}-2u_i^{k+1}+u^{k+1}_{i-1}+u^k_{i+1}-2u_i^k+u^k_{i-1})\quad,\quad i=1,\ldots,n
$$

whose stencil is shown below. 

In [ ]:
def plot_stencil_3():
    """Plot the stencil for the Crank-Nicolson method for the heat equation."""
    plt.close("stencil3")
    fig, ax = plt.subplots(num="stencil3")
    ax.plot(np.arange(3), np.zeros(3), "o", c="k")
    ax.plot(np.arange(3), np.ones(3), "o", c="k")
    ax.plot(np.arange(3), 2 * np.ones(3), "o", c="k")
    ax.text(-0.5, 0, "k-1", fontsize="xx-large")
    ax.text(-0.5, 1, "k", fontsize="xx-large")
    ax.text(-0.5, 2, "k+1", fontsize="xx-large")

    ax.text(0, -0.5, "i-1", fontsize="xx-large")
    ax.text(1, -0.5, "i", fontsize="xx-large")
    ax.text(2, -0.5, "i+1", fontsize="xx-large")

    ax.arrow(1, 1, 0, 0.75, fc="k", ec="k", head_width=0.10, head_length=0.25)

    ax.axis("off")

    ax.plot([0, 2], [2, 2], c="k")
    ax.plot([0, 2], [1, 1], c="k")
    ax.plot([1, 1], [1, 2], c="k")


plot_stencil_3()

This scheme, called the **Crank-Nicolson method**, is unconditionally stable and is second-order accurate in time as well as in space.

The greater stability of implicit finite difference methods enables them to take much larger time steps than are permissible with explicit methods, but they require more work per step because we must solve a system of equations at each step to determine the approximate solution values.

## Solving time-independent problems

Just as *time-dependent* parabolic and hyperbolic PDEs are analogous to *initial value problems* for ODEs, time-*independent* elliptic PDEs are analogous to *boundary-value problems* for ODEs, and most of the solution methods for ODE BVPs carry over to elliptic PDEs as well. 

For an elliptic boundary value problem, the solution at every point in the problem domain depends on all of the boundary data (in contrast to the limited domain of dependence for time-dependent problems), and consequently an approximate solution must be computed everywhere simultaneously, rather than being generated step by step using a recurrence, as in the previous examples.

Consequently, discretization of an elliptic boundary value problem results in a single system of algebraic equations to be solved for some finite-dimensional approximation to the solution.

### Laplace equation

The Laplace equation is a special case of the **Poisson equation**, which in two space dimensions has the form

$$u_{xx} + u_{yy} = f(x, y)$$

where $f$ is a given function defined on a domain whose boundary is typically a closed curve in $\mathbb{R}^2$, such as a square or circle. 

If $f \equiv 0$, then we have the **Laplace equation**.

There are numerous possibilities for the boundary conditions that must be specified on the boundary of the domain or portions thereof:

- **Dirichlet boundary conditions**, sometimes called essential boundary conditions, in which the solution $u$ is specified.
- **Neumann boundary conditions**, sometimes called natural boundary conditions, in which one of the derivatives $u_x$ or $u_y$ is specified.
- **Robin boundary conditions**, or **mixed boundary conditions**, in which a combination of solution values and derivative values is specified.

The Laplace equation models, for example, the electrostatic potential within a charge-free region given the potential on the boundary of the region. 

The Poisson equation models the electrostatic potential when there is also a known charge density within the region, represented by the function $f$.

For this reason, the Laplace equation or the Poisson equation is also sometimes called the **potential equation**.

> **Example: solving the Laplace equation with a finite difference method**
>
> Finite difference methods for elliptic boundary value problems proceed as we have seen before: we define a discrete mesh of points within the problem domain and replace the derivatives in the PDE by finite difference approximations, but then we seek a numerical solution at all of the mesh points simultaneously by solving a single system of algebraic equations.
>
>
> Consider the Laplace equation on the unit square
>
> $$u_{xx}+u_{yy}=0,\quad 0\leq x\leq 1,\quad 0\leq y \leq 1,$$
>
> We define a discrete mesh in the domain, including boundaries, as shown on the following figure, which also includes the boundary conditions. 

In [ ]:
def plot_laplace_mesh():
    """Plot the boundary conditions and mesh of the laplace equation example."""
    plt.close("laplace_mesh")
    fig, ax = plt.subplots(num="laplace_mesh")
    ax.plot(np.arange(1, 3), np.zeros(2), "o", c="k")
    ax.plot(np.arange(4), np.ones(4), "o", c="k")
    ax.plot(np.arange(4), 2 * np.ones(4), "o", c="k")
    ax.plot(np.arange(1, 3), 3 * np.ones(2), "o", c="k")

    ax.text(-0.5, 1.5, "0", fontsize="xx-large")
    ax.text(1.5, -0.5, "0", fontsize="xx-large")
    ax.text(1.5, 3.5, "1", fontsize="xx-large")
    ax.text(3.5, 1.5, "1", fontsize="xx-large")

    ax.arrow(0, 0, 0, 3.5, fc="k", ec="k", head_width=0.10, head_length=0.25)
    ax.arrow(0, 0, 3.5, 0, fc="k", ec="k", head_width=0.10, head_length=0.25)

    ax.axis("off")

    ax.plot([0, 3], [0, 0], c="k")
    ax.plot([0, 0], [0, 3], c="k")
    ax.plot([3, 3], [0, 3], c="k")
    ax.plot([0, 3], [3, 3], c="k")

    ax.text(0, 4, "y", fontsize="xx-large")
    ax.text(4, 0, "x", fontsize="xx-large")


plot_laplace_mesh()

>The interior grid points where we will compute the approximate solution are given by
>
> $$
(x_i,y_j)=(ih,jh), \quad i,j=1,\ldots,n
$$
>
>where in our example $n = 2$ and $h = 1/(n + 1) = 1/3$.
>
> Next we replace the second derivatives in the equation with the standard second-order centered difference
approximation at each interior mesh point to obtain the finite difference equations
>
> $$
\frac{u_{i+1,j}-2u_{i,j}+u_{i-1,j}}{h^2}+ \frac{u_{i,j-1}-2u_{i,j}+u_{i,j+1}}{h^2}=0\quad,\quad i,j=0,\ldots,n
$$
>
> where $u_{i,j}$ is an approximation to the true solution $u(x_i , y_j )$ and represents one of the given boundary values if $i$ or $j$ is $0$ or $n + 1$.
>
>Simplifying and writing out the resulting four equations explicitly, we obtain
>
> $$
\begin{split}
4u_{1,1}-u_{0,1}-u_{2,1}-u_{1,0}-u_{1,2} &= 0 \\
4u_{2,1}-u_{1,1}-u_{3,1}-u_{2,0}-u_{2,2} &= 0 \\
4u_{1,2}-u_{0,2}-u_{2,2}-u_{1,1}-u_{1,3} &= 0 \\
4u_{2,2}-u_{1,2}-u_{3,1}-u_{2,1}-u_{2,3} &= 0 \\
\end{split}
$$
>
> Writing these four equations in matrix form, we obtain
>
> $$
\mathbf{Ax}=\begin{bmatrix}
4&-1&-1&0\\
-1&4&0&-1\\
-1&0&4&-1\\
0&-1&-1&4\end{bmatrix}
\begin{bmatrix}u_{1,1}\\u_{2,1}\\u_{1,2}\\u_{2,2}\end{bmatrix}
=
\begin{bmatrix}
u_{0,1}+u_{1,0}\\
u_{3,1}+u_{2,0}\\
u_{0,2}+u_{1,3}\\
u_{3,2}+u_{2,3}\end{bmatrix}
=\begin{bmatrix}0\\0\\1\\1\end{bmatrix}=\mathbf{b}
$$
>
>This symmetric positive definite system of linear equations can be solved either by Cholesky factorization or by an iterative method, yielding the solution
>
>$$
\mathbf{x}=\begin{bmatrix}u_{1,1}\\u_{2,1}\\u_{1,2}\\u_{2,2}\end{bmatrix}=\begin{bmatrix}0.125\\0.125\\0.375\\0.375\end{bmatrix}
$$
>
> Note the symmetry in the solution, which reflects the symmetry in the problem, which we could have taken advantage of and solved a problem only half as large.

In a practical problem, the mesh size $h$ would need to be much smaller to achieve acceptable accuracy in the approximate solution of the PDE, and the resulting linear system would be much larger than in the preceding example. 

The matrix would be very sparse, however, since each equation would still involve at most only five of the variables, thereby saving substantially on work and storage.

In the next example you can change the parameter $N$ to define smaller mesh sizes. The sparsity of the matrix $A$ is visualized in the output, together with a 3D plot of the solution to the Laplace equation, using the same boundary conditions as in the minimal example above.

In [ ]:
def demo_laplace_equation():
    """Define and solve a laplace equation example with a finite-difference method.

    This generalizes the small problem we worked out analytically above.
    This code is partially copied from the poisson.py demo taught
    by Chris Rycroft in the Harvard Applied Math 205 course.
    """

    # define spatial discretizaiton
    N = 10
    # h = 1.0 / (N + 1)

    # define A
    A = np.zeros((N * N, N * N))

    for i in range(N):
        for j in range(N):
            ij = i + N * j

            A[ij, ij] = -4
            if i > 0.0:
                A[ij, ij - 1] = 1
            if i < N - 1:
                A[ij, ij + 1] = 1
            if j > 0.0:
                A[ij, ij - N] = 1
            if j < N - 1:
                A[ij, ij + N] = 1

    A *= -1

    # Display the sparsity structure of A
    print(A)
    plt.close("laplace_sparse")
    fig, ax = plt.subplots(num="laplace_sparse")
    ax.spy(A)
    ax.spines["top"].set_visible(True)
    ax.spines["bottom"].set_visible(False)
    ax.tick_params(axis="x", which="both", bottom=False)

    # define b, with boundary conditions as indictated in the book
    b = np.zeros(N * N)

    for i in range(N):
        for j in range(N):
            ij = i + N * j

            if i == 0.0:
                b[ij] = 0
            if i == N - 1:
                b[ij] = 1
            if j == 0.0:
                b[ij] = 0
            if j == N - 1:
                b[ij] = 1

    print(b)
    # solve the linear system
    u = np.linalg.solve(A, b)

    # --- Everything below makes an figure of the solution --

    uu = np.zeros((N + 2, N + 2))  # TODO add boundary conditions here
    for i in range(N):
        uu[i + 1, 1 : N + 1] = u[i * N : (i + 1) * N]
        uu[N + 1, :] = 1

    xa = np.linspace(0, 1, N + 2)
    mgx, mgy = np.meshgrid(xa, xa)
    plt.close("laplace")
    fig = plt.figure(num="laplace", clear=True)
    ax = fig.add_subplot(projection="3d")
    ax.plot_surface(mgx, mgy, uu, cmap=cm.plasma, rstride=1, cstride=1, linewidth=0)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

    print(uu)


demo_laplace_equation()